In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp

# Diffusion coefficients
diffusion_rate_1, diffusion_rate_2 = 0.1, 0.1
time_points = np.arange(0, 4 + 0.5, 0.5)  # Time evaluation points
reaction_rate = 1
simulation_time = 4
small_nu = 0.001
domain_length_x, domain_length_y = 20, 20
grid_points_x, grid_points_y = 64, 64
total_grid_points = grid_points_x * grid_points_y

def cheb(N):
    if N == 0: 
        differentiation_matrix = 0. 
        chebyshev_nodes = 1.
    else:
        n = np.arange(0, N+1)
        chebyshev_nodes = np.cos(np.pi * n / N).reshape(N+1, 1)
        coefficients = (np.hstack(([2.], np.ones(N-1), [2.])) * (-1)**n).reshape(N+1, 1)
        X = np.tile(chebyshev_nodes, (1, N+1))
        dX = X - X.T
        differentiation_matrix = np.dot(coefficients, 1. / coefficients.T) / (dX + np.eye(N+1))
        differentiation_matrix -= np.diag(np.sum(differentiation_matrix.T, axis=0))
    return differentiation_matrix, chebyshev_nodes.reshape(N+1)

# Define spatial grid
x_grid = np.linspace(-domain_length_x / 2, domain_length_x / 2, grid_points_x + 1)[:grid_points_x]
y_grid = np.linspace(-domain_length_y / 2, domain_length_y / 2, grid_points_y + 1)[:grid_points_y]
X, Y = np.meshgrid(x_grid, y_grid)

In [19]:
# Wavenumbers in Fourier space
wave_number_x = (2 * np.pi / domain_length_x) * np.concatenate((np.arange(0, grid_points_x / 2), np.arange(-grid_points_x / 2, 0)))
wave_number_x[0] = 1e-6  # Avoid division by zero
wave_number_y = (2 * np.pi / domain_length_y) * np.concatenate((np.arange(0, grid_points_y / 2), np.arange(-grid_points_y / 2, 0)))
wave_number_y[0] = 1e-6
KX, KY = np.meshgrid(wave_number_x, wave_number_y)
wavenumber_squared = KX**2 + KY**2

# Spiral wave initial conditions
num_spirals = 1
radius = np.sqrt(X**2 + Y**2)
angle = np.angle(X + 1j * Y)

u_initial = np.tanh(radius) * np.cos(num_spirals * angle - radius)
v_initial = np.tanh(radius) * np.sin(num_spirals * angle - radius)

# Transform to Fourier space
u_fft = fft2(u_initial)
v_fft = fft2(v_initial)

# Flatten into a single vector for ODE solver
initial_conditions = np.hstack([u_fft.reshape(total_grid_points), v_fft.reshape(total_grid_points)])

# Reaction-diffusion equations in Fourier space
def reaction_diffusion_rhs(t, uv_fft, grid_points_x, grid_points_y, total_grid_points, wavenumber_squared, reaction_rate):
    u_fft_vector = uv_fft[:total_grid_points]
    v_fft_vector = uv_fft[total_grid_points:]
    u_fft_matrix = u_fft_vector.reshape((grid_points_x, grid_points_y))
    v_fft_matrix = v_fft_vector.reshape((grid_points_x, grid_points_y))

    u_spatial = ifft2(u_fft_matrix)
    v_spatial = ifft2(v_fft_matrix)

    A = u_spatial**2 + v_spatial**2
    lambda_A = 1 - A
    omega_A = -reaction_rate * A

    rhs_u_fft = (-diffusion_rate_1 * wavenumber_squared * u_fft_matrix + fft2(lambda_A * u_spatial - omega_A * v_spatial)).reshape(total_grid_points)
    rhs_v_fft = (-diffusion_rate_2 * wavenumber_squared * v_fft_matrix + fft2(omega_A * u_spatial + lambda_A * v_spatial)).reshape(total_grid_points)

    return np.hstack([rhs_u_fft, rhs_v_fft])

# Solve the PDE system
solution = solve_ivp(
    reaction_diffusion_rhs,
    [0, simulation_time],
    initial_conditions,
    t_eval=time_points,
    args=(grid_points_x, grid_points_y, total_grid_points, wavenumber_squared, reaction_rate),
    method="RK45"
)

A1 = solution.y


In [20]:
# Chebyshev-based reaction-diffusion solver
N = 30
cheb_diff_matrix, cheb_nodes = cheb(N)
cheb_diff_matrix[N, :] = 0
cheb_diff_matrix[0, :] = 0
second_derivative_matrix = (np.dot(cheb_diff_matrix, cheb_diff_matrix)) / (10**2)

identity_matrix = np.eye(len(second_derivative_matrix))
laplacian_operator = np.kron(identity_matrix, second_derivative_matrix) + np.kron(second_derivative_matrix, identity_matrix)

# Define spatial grid
X, Y = np.meshgrid(cheb_nodes, cheb_nodes)
X, Y = X * 10, Y * 10

# Initial conditions
u_initial = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(num_spirals * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
v_initial = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(num_spirals * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))

uv_initial_conditions = np.hstack([u_initial.ravel(), v_initial.ravel()])

# Define ODE system
def reaction_diffusion_chebyshev(t, uv, grid_points, reaction_rate, laplacian_operator):
    u = uv[:grid_points]
    v = uv[grid_points:]
    A = u**2 + v**2
    lambda_A = 1 - A
    omega_A = -reaction_rate * A

    rhs_u = (diffusion_rate_1 * laplacian_operator @ u + (lambda_A * u - omega_A * v)).reshape(grid_points)
    rhs_v = (diffusion_rate_2 * laplacian_operator @ v + (omega_A * u + lambda_A * v)).reshape(grid_points)

    return np.hstack([rhs_u, rhs_v])

solution_cheb = solve_ivp(
    reaction_diffusion_chebyshev,
    [0, simulation_time],
    uv_initial_conditions,
    t_eval=time_points,
    args=(len(uv_initial_conditions) // 2, reaction_rate, laplacian_operator),
    method="RK45"
)

A2 = solution_cheb.y